In [1]:
# get multiclass data
# clean and normalize
# build network with optimizations
# test
# use real data

In [121]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

import keras.backend as K
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Input, BatchNormalization, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD, Adam

In [4]:
raw = pd.read_csv('C:/Users/Philippe/Dropbox/Desktop/datasets/bank-additional-full.csv', 
                 delimiter=';')

In [8]:
raw.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [9]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
age               41188 non-null int64
job               41188 non-null object
marital           41188 non-null object
education         41188 non-null object
default           41188 non-null object
housing           41188 non-null object
loan              41188 non-null object
contact           41188 non-null object
month             41188 non-null object
day_of_week       41188 non-null object
duration          41188 non-null int64
campaign          41188 non-null int64
pdays             41188 non-null int64
previous          41188 non-null int64
poutcome          41188 non-null object
emp.var.rate      41188 non-null float64
cons.price.idx    41188 non-null float64
cons.conf.idx     41188 non-null float64
euribor3m         41188 non-null float64
nr.employed       41188 non-null float64
y                 41188 non-null object
dtypes: float64(5), int64(5), object(11)
memory usa

In [10]:
raw.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,41188.00000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,40.02406,258.285010,2.567593,962.475454,0.172963,0.081886,93.575664,-40.502600,3.621291,5167.035911
std,10.42125,259.279249,2.770014,186.910907,0.494901,1.570960,0.578840,4.628198,1.734447,72.251528
min,17.00000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.00000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.00000,180.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.00000,319.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.00000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


In [12]:
raw.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')

In [19]:
raw['marital'].unique()

array(['married', 'single', 'divorced', 'unknown'], dtype=object)

In [ ]:
# y, job

In [31]:
num_cols = ['age', 'duration', 'campaign', 'pdays', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx',
         'euribor3m', 'nr.employed']

cat_cols = ['age', 'marital', 'education', 'default', 'housing', 'loan', 'contact']

In [32]:
df_num = raw[num_cols]

In [33]:
df_num.head()

,age,duration,campaign,pdays,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,56,261,1,999,1.1,93.994,-36.4,4.857,5191.0
1,57,149,1,999,1.1,93.994,-36.4,4.857,5191.0
2,37,226,1,999,1.1,93.994,-36.4,4.857,5191.0
3,40,151,1,999,1.1,93.994,-36.4,4.857,5191.0
4,56,307,1,999,1.1,93.994,-36.4,4.857,5191.0


In [38]:
sc = preprocessing.StandardScaler()
X_nums = sc.fit_transform(df_num)

C:\Users\Philippe\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Philippe\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [40]:
df_cat = raw[cat_cols]

In [71]:
df_cat.values

(41188, 7)

In [83]:
enc = preprocessing.OneHotEncoder()
enc.fit(df_cat.values)
X_cats = enc.transform(df_cat.values).toarray()

In [65]:
# enc = preprocessing.OneHotEncoder()
# X = [['Male', 1], ['Female', 3], ['Female', 2]]
# enc.fit(X)
# labels = enc.transform(X).toarray()

In [84]:
print('num shape', X_nums.shape)
print('cat shape', X_cats.shape)

num shape (41188, 9)
cat shape (41188, 101)


In [85]:
X = np.concatenate((X_nums, X_cats), axis=1)

In [86]:
X.shape

(41188, 110)

In [101]:
y_cat = raw['y']
y_job = raw['job']

In [95]:
enc.fit(y_cat.values.reshape(41188, 1))
y1 = enc.transform(y_cat.values.reshape(41188, 1)).toarray()

In [102]:
enc.fit(y_job.values.reshape(41188, 1))
y_job = enc.transform(y_job.values.reshape(41188, 1)).toarray()

In [127]:
Y = np.concatenate((y1, y_job), axis=1)

In [128]:
Y.shape

(41188, 14)

### Single Category, Multiclass Model

In [109]:
X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size = 0.3)

In [110]:
X_train.shape

(28831, 110)

In [122]:
K.clear_session()

model = Sequential()
model.add(Dense(110, kernel_initializer='he_normal', activation='relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(75, kernel_initializer='he_normal', activation='relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(55, kernel_initializer='he_normal', activation='relu'))
model.add(BatchNormalization())

model.add(Dense(25, kernel_initializer='he_normal', activation='relu'))
model.add(BatchNormalization())

model.add(Dense(10, kernel_initializer='he_normal', activation='relu'))
model.add(BatchNormalization())

model.add(Dense(2, activation='softmax'))

model.compile(Adam(lr=0.1), 'categorical_crossentropy', metrics=['accuracy'])

In [123]:
model.fit(X_train, y_train, batch_size=64, epochs=10, verbose=1, validation_split=0.1)

Train on 25947 samples, validate on 2884 samples
Epoch 1/10
25947/25947 [==============================] - 5s 175us/step - loss: 0.2420 - acc: 0.8933 - val_loss: 0.2587 - val_acc: 0.8922
Epoch 2/10
25947/25947 [==============================] - 2s 86us/step - loss: 0.2217 - acc: 0.8989 - val_loss: 0.1937 - val_acc: 0.9046
Epoch 3/10
25947/25947 [==============================] - 2s 86us/step - loss: 0.2202 - acc: 0.8969 - val_loss: 0.2363 - val_acc: 0.8936
Epoch 4/10
25947/25947 [==============================] - 2s 86us/step - loss: 0.2148 - acc: 0.9020 - val_loss: 0.2162 - val_acc: 0.8970
Epoch 5/10
25947/25947 [==============================] - 2s 86us/step - loss: 0.2144 - acc: 0.8998 - val_loss: 0.1891 - val_acc: 0.9161
Epoch 6/10
25947/25947 [==============================] - 2s 87us/step - loss: 0.2159 - acc: 0.8991 - val_loss: 0.1919 - val_acc: 0.9102
Epoch 7/10
25947/25947 [==============================] - 2s 85us/step - loss: 0.2077 - acc: 0.9038 - val_loss: 0.1954 - val_acc

In [124]:
results = model.evaluate(X_test, y_test)

12357/12357 [==============================] - 0s 31us/step


### Multiclass and Multicategory Model

In [134]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3)

In [137]:
K.clear_session()

model = Sequential()
model.add(Dense(110, kernel_initializer='he_normal', activation='relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(75, kernel_initializer='he_normal', activation='relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(55, kernel_initializer='he_normal', activation='relu'))
model.add(BatchNormalization())

model.add(Dense(25, kernel_initializer='he_normal', activation='relu'))
model.add(BatchNormalization())

model.add(Dense(10, kernel_initializer='he_normal', activation='relu'))
model.add(BatchNormalization())

model.add(Dense(14, activation='softmax'))

model.compile(Adam(lr=0.1), 'binary_crossentropy', metrics=['accuracy'])

In [138]:
model.fit(X_train, Y_train, batch_size=64, epochs=10, verbose=1, validation_split=0.1)

Train on 25947 samples, validate on 2884 samples
Epoch 1/10
25947/25947 [==============================] - 5s 188us/step - loss: 0.2831 - acc: 0.9009 - val_loss: 0.2947 - val_acc: 0.8827
Epoch 2/10
25947/25947 [==============================] - 2s 89us/step - loss: 0.2689 - acc: 0.9021 - val_loss: 0.2664 - val_acc: 0.8944
Epoch 3/10
25947/25947 [==============================] - 2s 88us/step - loss: 0.2655 - acc: 0.9024 - val_loss: 0.2594 - val_acc: 0.8853
Epoch 4/10
25947/25947 [==============================] - 2s 87us/step - loss: 0.2647 - acc: 0.9027 - val_loss: 0.2635 - val_acc: 0.9027
Epoch 5/10
25947/25947 [==============================] - 2s 88us/step - loss: 0.2635 - acc: 0.9024 - val_loss: 0.2550 - val_acc: 0.9095
Epoch 6/10
25947/25947 [==============================] - 2s 87us/step - loss: 0.2633 - acc: 0.9027 - val_loss: 0.2623 - val_acc: 0.9013
Epoch 7/10
25947/25947 [==============================] - 2s 83us/step - loss: 0.2623 - acc: 0.9034 - val_loss: 0.2582 - val_acc

In [139]:
model.evaluate(X_test, Y_test)

12357/12357 [==============================] - 0s 39us/step


[0.26168004600585176, 0.9091260466576009]

In [140]:
Y_test_pred = model.predict(X_test)

In [147]:
Y_test_pred[13, :]

array([0.38433847, 0.1220438 , 0.27223632, 0.01038784, 0.01524288,
       0.00119365, 0.02540438, 0.00044184, 0.02034087, 0.02291964,
       0.02192678, 0.09533298, 0.00691554, 0.0012751 ], dtype=float32)

In [148]:
Y_test[13, :]

array([0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])